In [167]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import matplotlib.pyplot as plt
import string
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
#stopwords = stopwords.words('english')
import gensim
import nltk
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
import warnings
from scipy.optimize import linear_sum_assignment
warnings.filterwarnings('ignore')

In [168]:
with open("./data/linux/linux_data_clean.txt", "r") as read_file:
    data = json.load(read_file)

In [171]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


In [172]:
def cal_param_sim_basic(comd1, comd2):
    par_names_comd1 = []
    par_names_comd2 = []
    stop_words = stopwords.words('english')
    for key_comd1,val_comd1 in comd1.items():
        par_names_comd1.append(key_comd1)
    for key_comd2, val_comd2 in comd2.items():
        par_names_comd2.append(key_comd2)
    comds_pars_comps = pd.DataFrame(columns=par_names_comd1, index=par_names_comd2)
    for key1, val1 in comd1.items():
        val1 = " ".join(word for word in val1.split() if word.lower() not in stop_words)
        for key2, val2 in comd2.items():
            val2 = " ".join(word for word in val2.split() if word.lower() not in stop_words)
            if len(val1)>2 and len(val2)>2:
                comds_pars_comps.loc[key2, key1] = round(cosine_sim(val1, val2),3)
                
            else:
                 comds_pars_comps.loc[key2, key1] = 0
    #comds_pars_comps = comds_pars_comps.loc[~(comds_pars_comps==0).all(axis=1)]
    return comds_pars_comps
        

In [173]:
def cal_linear_sum_assignment(data_matrix):
    data_matrix_array = data_matrix.to_numpy(dtype=float)
    row_ind, col_ind = linear_sum_assignment(data_matrix_array)
    return data_matrix_array[ row_ind,col_ind].sum()

In [174]:
def cal_par_sim_matrix(first_comd, second_comd):
    sim_matrix = cal_param_sim_basic(first_comd, second_comd)  
    #sim_score = unique_pars_sim_score(sim_matrix)
    sim_score = -cal_linear_sum_assignment(-sim_matrix)/min(sim_matrix.shape)
    return sim_score
    

In [175]:
def all_pars_sim_score(df):
    #df = df.loc[~(df==0).all(axis=1)]
    #df = df.loc[:, (df != 0).any(axis=0)]
    cols = df.columns
    indx = df.index
    max_sim = 0
    for c in cols:
        max_sim+=df[c].max()
    for i in indx:
        max_sim+=df.loc[i].max()
    if max_sim>0:
        sim_score = round(max_sim/(len(cols)+len(indx)),3)
    else:
        sim_score=0
    return sim_score


In [176]:
def common_pars_sim_score(df):
    #df = df.loc[~(df==0).all(axis=1)]
    #df = df.loc[:, (df != 0).any(axis=0)]
    cols = df.columns
    indx = df.index
    max_sim = 0
    com_flags = 0
    for c in cols:
        for i in indx:
            if c == i:
                com_flags+=1
                max_sim+=df.loc[c, i]
    if com_flags>0:
        sim_score = round(max_sim/com_flags,3)
    else:
        sim_score= 0
    return sim_score

In [177]:
def unique_pars_sim_score(df):
    #df = df.loc[~(df==0).all(axis=1)]
    #df = df.loc[:, (df != 0).any(axis=0)]
    cols = list(df.columns)
    indx = list(df.index)
    max_sim = 0
    for c in cols:
        for i in indx:
            if c == i:
                max_sim+=df.loc[c, i]
    if max_sim>0:
        sim_score = round(max_sim/len(set(cols+indx)),3)
    else:
        sim_score= 0
    return sim_score

In [178]:
def create_pars_sim_df(data):
    columns_names = []
    for dc in range(len(data['commands'])):
        columns_names.append(data['commands'][dc]['commands'])
    par_sim_rows_cols = pd.DataFrame(columns=columns_names, index=columns_names)
    for col1 in range(len(data['commands'])):
        comd1_name = data['commands'][col1]['commands']
        first_comd = data['commands'][col1]['parameters']
        if len(first_comd)==1 and 'None' in first_comd:
            for row1 in range(len(data['commands'])):
                comd2_name = data['commands'][row1]['commands']
                par_sim_rows_cols.loc[comd1_name, comd2_name]=0
        else:
            for row1 in range(len(data['commands'])):
                comd2_name = data['commands'][row1]['commands']
                second_comd = data['commands'][row1]['parameters'] 
                #print(col1, row1, comd1_name, comd2_name)
                if len(second_comd)==1 and 'None' in second_comd:
                    par_sim_rows_cols.loc[comd1_name, comd2_name]=0
                else:
                    print(col1, row1, comd1_name, comd2_name)
                    sim_score_two_cmds = cal_par_sim_matrix(first_comd, second_comd)
                    par_sim_rows_cols.loc[comd1_name, comd2_name]=sim_score_two_cmds
    #par_sim_rows_cols.to_csv('./data/linux/my_csv.csv', mode='a', header=False)
    return par_sim_rows_cols
            

In [179]:
def extract_pair_sim_pars(data1, data2):
    par_key = []
    par_val = []
    first_cmd = []
    second_cmd = []
    
    for k1, v1 in data1.items():
        par_key.append(k1)
        first_cmd.append(k1)
        par_val.append(v1)
    for k2, v2 in data2.items():
        par_key.append(k2)
        second_cmd.append(k2)
        par_val.append(v2)
               
    
    #vect = TfidfVectorizer(min_df=1, stop_words="english" ) #, stop_words="english"  
    vect = TfidfVectorizer(tokenizer=normalize,min_df=1, stop_words='english')
    tfidf = vect.fit_transform(par_val)                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T 
    pair_sim = pairwise_similarity.toarray()
    sim_df = pd.DataFrame(pair_sim, columns=par_key, index=par_key)
    sim_df = sim_df.loc[second_cmd, first_cmd]
    if first_cmd==second_cmd:
        sim_df = sim_df.loc[:,~sim_df.columns.duplicated()]
        sim_df = sim_df.drop_duplicates()

    return sim_df

In [139]:
#par_sim_linear_sum_assign = create_pars_sim_df(data)

In [140]:
#par_sim_linear_sum_assign.to_csv('./data/linux/linux_pars_sim_linear_sum_assign.csv')

In [187]:
d1 = data['commands'][1231]['parameters']
d2 = data['commands'][375]['parameters']
#test_df = cal_param_sim_basic(d1, d2)
test_df1 = extract_pair_sim_pars(d1, d2)
test_df1

,--output=MODE,"--service=SERVICE[:SERVICE...],-sSERVICE:SERVICE...",--with-nss=BOOL,--synthesize=BOOL,-N,--no-pager,--no-legend
-a,0.021400,0.045263,0.025661,0.004795,0.047041,0.000000,0.0
-A,0.007766,0.000000,0.018928,0.006157,0.035878,0.000000,0.0
-d,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
-l,0.000000,0.000000,0.000000,0.000000,0.056584,0.000000,0.0
-f,0.000000,0.000000,0.000000,0.000000,0.055369,0.000000,0.0
-F,0.000000,0.000000,0.000000,0.000000,0.055369,0.000000,0.0
-q,0.000000,0.000000,0.000000,0.000000,0.053069,0.000000,0.0
-n,0.015760,0.000000,0.000000,0.021256,0.075606,0.000000,0.0
"--window=<n>,--depth=<n>",0.009139,0.000000,0.022276,0.008991,0.034408,0.000000,0.0
--threads=<n>,0.000000,0.000000,0.000000,0.034903,0.158471,0.000000,0.0


In [188]:
-cal_linear_sum_assignment(-test_df)/min(test_df.shape)

0.4706190476190476

In [189]:
-cal_linear_sum_assignment(-test_df1)/min(test_df1.shape)

0.06336401568794094

In [ ]:
par_sim_linear_sum_assign[par_sim_linear_sum_assign == -0]= 0

In [ ]:
for col in par_sim_linear_sum_assign.columns:
    if par_sim_linear_sum_assign[col][col]>0:
        par_sim_linear_sum_assign[col][col]=1

In [ ]:
par_sim_unique_rows_cols = pd.read_csv('./data/pars_sim_common_rows_cols.csv', index_col=0)

In [ ]:
par_sim_unique_rows_cols

In [ ]:
#par_sim_unique_rows_cols.to_csv('./data/pars_sim_common_rows_cols.csv')

In [ ]:
par_sim_all_rows_cols = pd.read_csv('./data/pars_sim_all_rows_cols.csv', index_col=0)

In [ ]:
par_sim_all_rows_cols

In [ ]:
id_ar = np.identity((len(par_sim_linear_sum_assign.columns)))
par_sim_linear_sum_assign[par_sim_linear_sum_assign == -0]= 0
temp = par_sim_linear_sum_assign.round(3) - id_ar
#temp[temp < 0] = 0
temp2 = temp[temp.idxmax()]
a = temp.max()
b = temp.idxmax()
c = dict(a)
d = dict(b)
sim_sc = []
for k, v in c.items():
    sim_sc.append([k, d[k], c[k]] )
sim_sc_pd = pd.DataFrame(sim_sc, columns=['first', 'second', 'score'])
asc = sim_sc_pd.sort_values('score', ascending=False)
figure = plt.figure(figsize=(10, 8))
plt.plot(asc['first'], asc['score']);

In [ ]:
c = cal_param_sim_basic(data['commands'][689]['parameters'],data['commands'][689]['parameters'])

In [ ]:
'''for col in aa.columns:
    if col not in none_pars:
        if aa.loc[col,col]<1:
            print(col, aa.loc[col,col])
            aa.loc[col,col] = 1
            print(col, aa.loc[col,col])
      
aa.loc[:, (aa == 0).all()] '''          

In [ ]:
tot_non = 0
for i in range(len(data['commands'])):
    parameters = data['commands'][i]['parameters']
    if 'None' in parameters:
          tot_non+=1
tot_non

In [ ]:
par_list =['sub-command','option', 'paramreter','parameters','term', 'commands', 'subcommand', 'base parameters','option flag']

In [ ]:
df = pd.read_csv('./data/pars_sim_linear_sum_assign.csv', index_col=0)

In [ ]:
df[df<0]=0
df[df == -0 ]=0

In [ ]:
id_ar = np.identity((len(df.columns)))
temp = df.round(3) - id_ar
temp2 = temp[temp.idxmax()]
a = temp.max()
b = temp.idxmax()
c = dict(a)
d = dict(b)
sim_sc = []
for k, v in c.items():
    sim_sc.append([k, d[k], c[k]] )
sim_sc_pd = pd.DataFrame(sim_sc, columns=['first', 'second', 'score'])
asc = sim_sc_pd.sort_values('score', ascending=False)
figure = plt.figure(figsize=(10, 8))
asc = asc[asc['second']!='cls']
plt.plot(asc['first'], asc['score']);

In [ ]:
asc#[asc['second']!='cls']